In [1]:
import numpy as np
import pandas as pd
import copy

In [2]:
table=pd.read_csv('names.csv')
names=dict(np.array(table))

In [20]:
algebra_operators={-1:'+',-2:'-',-3:'*',-4:'/'}
binar_operators={-10:'='}
cond_op={-20:'>0',-21:'<0',-22:'==0',-23:'>=0',-24:'<=0'}
unar_operators={-5:'-',-6:'print',-7:'goto'}
operands={1:[1,'numb',1],2:[2,'numb',2],3:[3,'numb',3],4:[4.5,'numb',4.5],
          5:['some_string','char','some_string'],6:['a','var',np.NaN],7:['b','var',np.NAN],8:['c','mark',11]}

In [25]:
#stack=[6,2,2,6,-5,-1,-3,-6,-100]
#stack=[7,3,-10,6,7,7,-3,-10,6,-6,-100]# '7'=3;'6'='7'*'7';print('6')
stack=[7,3,-10,7,8,-20,6,7,7,-3,-10,6,-6,-100]
#stack=[6,7,1,-7,-7,6,-6,-100]

In [26]:
class parser(object):
    def __init__(self,stack,operands,trace=True):
        self.operands=copy.deepcopy(operands)
        self.stack=stack
        self.count_stack=[]
        self.end_symbols=[-100]
        self.trace=trace
    def show(self):
        print(self.operands)
    def count_reversed_form(self,start_point=0):
        stack=self.stack
        i=0
        while stack[i] not in self.end_symbols:
            if self.trace==True: print(i,self.count_stack)
            if stack[i] in self.operands.keys():
                self.count_stack.append([stack[i],'link'])                        
            else:
                if stack[i] in algebra_operators:
                    operation=algebra_operators[stack[i]]
                    to_eval=self.unpack(number=2)
                    types=self.get_type(to_eval)
                    if types[0]=='numeric' and types[1]=='numeric':
                        oper_result=eval(str(to_eval[0])+operation+str(to_eval[1]))
                        #print(oper_result)
                        self.count_stack[-2]=[oper_result,'value']
                        self.count_stack=self.count_stack[:-1]
                    else:
                        print('ERROR: wrong types')
                        return -1
                if stack[i] in binar_operators:
                    values=self.unpack(number=2)
                    types=self.get_type(values)
                    if stack[i]==-10:
                        if isnan(types[0]) and isnan(types[1]):
                            print('ERROR: both operands are undefined')
                            return -2
                        elif types[0]==types[1] or isnan(types[0]):
                            self.operands[self.count_stack[-2][0]][2]=values[1]
                            self.count_stack[-2]==[values[1],'value']
                            self.count_stack=self.count_stack[:-1]
                        else:
                            print('ERROR: worong types, trying to write {}->{}'.format(values[1],types[0]))
                            return -2
                if stack[i] in unar_operators:
                    value=self.unpack(number=1)
                    if stack[i]==-5 and self.get_type(value):
                        self.count_stack[-1]=[value*-1,'value']
                    if stack[i]==-6:
                        print(value)
                        self.count_stack=self.count_stack[:-1]
                    if stack[i]==-7:
                        adress=self.unpack(number=1)
                        self.count_stack=self.count_stack[:-1]
                        i=adress
                        continue
                if stack[i] in cond_op:
                    operator=cond_op[stack[i]]
                    values=self.unpack(number=2) ## value+mark
#                     print(str(values[0])+operator)
                    result=eval(str(values[0])+operator)  ## condition on value
                    if result: ## go further
                        i+=1
                        self.count_stack=self.count_stack[:-2]
                        continue
                    else: # go on mark
                        i=values[1]
                        self.count_stack=self.count_stack[:-2]
                        continue
            i+=1
        return self.operands
    def unpack(self,number=2):
        result=[]
        if number==2:
            for i in self.count_stack[-2:]:
                if i[1]=='link':
                    result.append(self.operands[i[0]][2])
                    continue
                elif i[1]=='value':
                    result.append(i[0])
                    continue
                else:
                    print('Unknown type')
            #print(result)
            return result
        if number==1:
            temp=self.count_stack[-1]
            return (self.operands[temp[0]][2] if self.count_stack[-1][1]=='link' else temp[0])
    def get_type(self,x):
        a=x if isinstance(x,(list,tuple)) else [x]
        to_ret=[]
        for i in a:
            if isnan(i):
                to_ret.append(np.NAN)
            else:
                if isinstance(i, (int, long, float)):
                    to_ret.append('numeric')
                else:
                    to_ret.append('str')
        return(to_ret if len(to_ret)>1 else to_ret[0])
def isnan(x):
    return x!=x

In [27]:
worker=parser(stack,operands,True)

In [28]:
worker.count_reversed_form(stack)

(0, [])
(1, [[7, 'link']])
(2, [[7, 'link'], [3, 'link']])
(3, [[7, 'link']])
(4, [[7, 'link'], [7, 'link']])
(5, [[7, 'link'], [7, 'link'], [8, 'link']])
3>0
(6, [[7, 'link']])
(7, [[7, 'link'], [6, 'link']])
(8, [[7, 'link'], [6, 'link'], [7, 'link']])
(9, [[7, 'link'], [6, 'link'], [7, 'link'], [7, 'link']])
(10, [[7, 'link'], [6, 'link'], [9, 'value']])
(11, [[7, 'link'], [6, 'link']])
(12, [[7, 'link'], [6, 'link'], [6, 'link']])
9


{1: [1, 'numb', 1],
 2: [2, 'numb', 2],
 3: [3, 'numb', 3],
 4: [4.5, 'numb', 4.5],
 5: ['some_string', 'char', 'some_string'],
 6: ['a', 'var', 9],
 7: ['b', 'var', 3],
 8: ['c', 'mark', 11]}

In [112]:
operands

{1: [1, 'numb', 1],
 2: [2, 'numb', 2],
 3: [3, 'numb', 3],
 4: [4.5, 'numb', 4.5],
 5: ['some_string', 'char', 'some_string'],
 6: ['a', 'var', nan],
 7: ['b', 'var', nan]}

In [ ]:
a='dsfsdf'

In [ ]:
a=[[1,'a'],[2,'b'],[3,'c']]

In [ ]:
for i in a[-2:]:
    print(i)

In [ ]:
print(names)

In [ ]:
## 1 с перезаписью, заполнение при объявлении
# stack=[-5,-10,101,-7,1,2,-1,4,-4,3,-2,-5,101,-7,1,2,3,-4,4,-4,-1,5,6,-2,7,8,-4,9,-1,-4,-1,-5]

## 1 переменная, заполнение после объявления
#stack=[-5,-10,101,-5,101,-7,1,2,-1,4,-4,3,-2,-5]

## С goto
# stack=[-5,-10,101,-7,1,2,-1,4,-4,3,-2,-5,  
#        -21,30,101,-7,1,2,3,-4,4,-4,-1,5,6,-2,7,8,-4,9,-1,-4,-1,-5,30,-5,
#        101,-7,1,2,-1,6,-4,3,-2,-5]

## 2 переменные с ошибкой типов
# stack=[-5,-10,101,-7,1,2,-1,4,-4,3,-2,-5,-10,102,-7,101,10,3,-4,4,-4,-1,5,6,-2,7,8,-4,9,-1,-4,-1,-5] 

## 2 переменные заполнение при объявлении
stack=[-5,-10,101,-7,1,2,-1,4,-4,3,-2,-5,-10,102,-7,101,2,3,-4,4,-4,-1,5,6,-2,7,8,-4,9,-1,-4,-1,-5]

In [ ]:
table.tail(14)

In [ ]:
worker.interpretate()

In [ ]:
a=2.5

In [ ]:
b = str(type(a))

In [ ]:
b.split("'")[1]

In [ ]:
b.find("'")

In [ ]:
b.__repr__

## (1+2)*4-3 <=> 1 2 + 4 × 3 -

In [ ]:
count_reversed_form(stack,start_point=0)

In [ ]:
a=[1,2,-3,4,5]

In [ ]:
exec(expression)

In [ ]:
expression=str(names[1])+str(names[-1])+str(names[3])

In [ ]:
a=exec(str('1')+str('+')+str('3'))

In [ ]:
expression

In [ ]:
def interpretate(self):
        stack=self.stack
        #print(stack)
        i=0
        while i!=len(stack):
            #print(stack[i])
            if stack[i]>100:
                ## Work with variables
                #print(stack[i:])
                if stack[i-1]>-20 and stack[i-1]<=-10: ## First declaration(type before variable)
                    #print(stack[i:])
                    if self.find_value(stack[i]):  ## Check for redeclaration
                        print("ERROR: {} variable redeclaration".format(names[stack[i]]))
                        return -3
                    variable={}
                    variable['type']=names[stack[i-1]]
                    variable['cord']=stack[i]
                    if stack[i+1] not in self.end_symbols: ## some computations just after declaration
                        variable['value'],i=self.count_reversed_form(start_point=i+2)
                        if self.get_type(variable['value'])!=variable['type']:
                            print('ERROR: types mismatch:{}->{}'
                                  .format(self.get_type(variable['value']),variable['type']))
                            return -4
                        else:
                            self.user_variables.append(variable)
                        #print(self.user_variables)
                    else: ## no operations after declaration
                        variable['value']=np.NAN
                        self.user_variables.append(variable)
                else :  ## variable was already declared
                    declared_variable=self.find_value(stack[i])
                    if not declared_variable:
                        print('ERROR: variable {} was not declared'.format(names[stack[i]]))
                        return -5
                    new_value,temp=self.count_reversed_form(start_point=i+2)
                    if self.get_type(new_value)!=declared_variable['type']:
                            print('ERROR: types mismatch:{}->{}'
                                  .format(self.get_type(new_value),declared_variable['type']))
                            return -4
                    else:
                        self.change_value(stack[i],new_value)
                        i=temp
                    #print(self.user_variables)
                i+=1
                #print(stack[i])
                continue
            if stack[i]==-8:  ## goto mark
                self.check_points[stack[i+1]]=i+1
                continue
            if stack[i]==-21: ## goto
                if stack[i+1] in self.check_points.keys():
                    i=self.check_points[stack[i]]
                    continue
                else:
                    temp=stack[i+1]
                    try:
                        i=next(x for x in range(i+2,len(stack)) if stack[x]==temp)+1
                        continue
                    except:
                        print('ERROR: no mark for goto {} found'.format(names[stack[i+1]]))
#             if stack[i]==-31:
                
            i+=1
        return self.user_variables
    def find_value(self,cord):
        for i in self.user_variables:
            if i['cord']==cord:
                return i
        return False
    def change_value(self,cord,new_value):
        for i in self.user_variables:
            if i['cord']==cord:
                i['value']=new_value
                return 0
    def get_type(self,x):
        temp=str(type(x))
        return temp.split("'")[1]